<a href="https://colab.research.google.com/github/coleterrell97/portfolio/blob/master/ML_Practice/digit_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.optim as O
from google.colab.patches import cv2_imshow
EPOCHS = 1000

In [ ]:
def reshape_labels(labels):

  a = labels.numpy()
  b = np.zeros((a.size, int(a.max()+1)))
  b[np.arange(a.size),a.astype("int")] = 1
  return b.astype("float32")

In [ ]:
df = pd.read_csv("train.csv")
train_data_np = df.to_numpy(dtype="float32")
train_data = torch.from_numpy(train_data_np)
features = train_data[:,1:]
labels = train_data[:,0].type(torch.LongTensor)
# labels = torch.from_numpy(reshape_labels(train_data[:,0]))
train_ds = TensorDataset(features, labels)
train_dl = DataLoader(train_ds, batch_size=2048, shuffle=True)


In [ ]:
class Net(nn.Module):
  def __init__(self, inputs, hidden_layer_size, outputs):
    super(Net, self).__init__()
    self.input_layer = nn.Linear(inputs,hidden_layer_size)
    self.hidden_layer = nn.Linear(hidden_layer_size, outputs)

  def forward(self, x):
    x = F.relu(self.input_layer(x))
    x = self.hidden_layer(x)
    return x

In [ ]:
digit_recognizer = Net(784, 200, 10)

In [ ]:
loss_fn = F.cross_entropy
optimizer = O.SGD(digit_recognizer.parameters(), lr=1e-04)

In [ ]:
def fit(epochs, dl, model, loss_fn, opt):
  loss = 0
  for epoch in range(epochs):
    for xb, yb in dl:
      predictions = model(xb)
      loss = loss_fn(predictions,yb)
      loss.backward()
      opt.step()
      opt.zero_grad()
  print(loss)

In [ ]:
fit(EPOCHS, train_dl, digit_recognizer, loss_fn, optimizer)



tensor(0.0275, grad_fn=<NllLossBackward>)


In [ ]:
df = pd.read_csv("test.csv")
test_data_np = df.to_numpy(dtype="float32")
test_data = torch.from_numpy(test_data_np)

In [ ]:
predictions = F.softmax(digit_recognizer(test_data), dim=1)
probs, predictions = torch.max(predictions, dim=1)
for prediction_index in range(len(predictions)):
  cv2_imshow(test_data[prediction_index].numpy().reshape(28,28))
  print(predictions[prediction_index])
